# *technische Voraussetzungen*

In [ ]:
# medienfrech'sche IT-Abteilung, 2020-11-21, 2020-11-30
%pip install plotly pillow pandas pip-tools
!pip3 freeze > requirements.txt

# Bild laden

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from PIL import Image
import colorsys

!curl "..." > a.jpg

img = Image.open('a.jpg').convert('RGB')
img.thumbnail((64, 48), Image.ANTIALIAS)
pix = img.load()
w, h = img.getdata().size

# Bittiefe verringern
farb_bittiefe = lambda v: v
pixel_bittiefe = lambda rgb: tuple(map(farb_bittiefe, rgb))
for x in range(w):
    for y in range(h):
        pix[x, y] = pixel_bittiefe(pix[x, y])

# img, pix, w, h

In [ ]:
img = Image.new('RGB', (256*4, 16*4))
pix = img.load()
w, h = img.getdata().size
for y in range(h):
    for x in range(w):
        pix[x, y] = tuple(round(v*255) for v in colorsys.hls_to_rgb((x-y)/1000, 0.5, 1))

px.imshow(img)

# Helligkeitswerte als Höhe interpretieren

In [ ]:
def sub_figure(pix_func, rgb_func):
    return go.Surface(
        z=[[pix_func(x, y) for x in range(w)] for y in range(h)],
        colorscale=[[l/0xff, rgb_func(l)] for l in range(0x100)])


figRGB = make_subplots(rows=1, cols=3, specs=[[{'type': 'surface'} for _ in range(3)]])
figRGB.add_trace(sub_figure(lambda x, y: pix[x, y][0]      , lambda l: f'rgb({l}, 0, 0)'), row=1, col=1)
figRGB.add_trace(sub_figure(lambda x, y: pix[x, y][1]      , lambda l: f'rgb(0, {l}, 0)'), row=1, col=2)
figRGB.add_trace(sub_figure(lambda x, y: pix[x, y][2]      , lambda l: f'rgb(0, 0, {l})'), row=1, col=3)

fig = go.Figure(data=sub_figure(lambda x, y: sum(pix[x, y]) // 3, lambda l: f'rgb({l}, {l}, {l})'))

fig.show()
figRGB.show()

# Helligkeitswerte positionsambivalent aufzeichnen

In [ ]:
grauwert = lambda x, y: (sum(pix[x, y]) // 3) &0b1100_0000

histogram = [len([None for x in range(w) for y in range(h) if int(grauwert(x, y)) == k])/w/h for k in range(256)]
fig = go.Figure([go.Bar(x=list(range(256)), y=histogram)])
fig.show()

In [ ]:


plural = lambda n: 's' * (abs(n) != 1)

#for y in range(h):
#    for x in range(w):
#        pix[x, y] = (sum(pix[x, y]) // 3,) * 3

def reduziere(v, number_of_colors=256):
    if number_of_colors <= 1:
        return 0
    v2 = int(round(v * (number_of_colors-1) / 255))
    v3 = v2 * 255 / (number_of_colors-1)
    return int(round(v3))
    
        
#bit_depth_fig = make_subplots(1, 2)
bit_depth_fig = go.Figure()

images = []
for number_of_colors in map(lambda bits: 1 << bits, range(1, 8+1)):
    images.append(img.copy())
    pixels = images[-1].load()
    for y in range(h):
        for x in range(w):
            pixels[x, y] = tuple(reduziere(v, number_of_colors=number_of_colors) for v in pixels[x, y])
    bit_depth_fig.add_trace(go.Image(z=images[-1]))
    #subplot = make_subplots(rows=1, cols=2)
    #subplot.add_trace(go.Image(z=images[-1]), row=1, col=1)
    #bit_depth_fig.add_trace(go.Image(z=images[-1]), row=1, col=1)
    #bit_depth_fig.add_trace(go.Image(z=images[-1]), row=1, col=2)

steps = []
for j, _ in enumerate(bit_depth_fig.data):
    step = dict(
        method='update',
        args=[dict(visible=([False] * len(bit_depth_fig.data)), title='slider!')],
        label=f'{j+1!r} Bit{plural(j+1)}',
    )
    step['args'][0]['visible'][j] = True
    steps.append(step)
    
bit_depth_fig.data[-1].visible = True
    
sliders = [dict(
    active=len(bit_depth_fig.data)-1,
    currentvalue={'prefix': 'Bittiefe: '},
    pad={'t': 3},
    steps=steps,
)]
    
bit_depth_fig.update_layout(sliders=sliders)
bit_depth_fig.show()

# TODO dropdown menu ; possible a histogram?

# Experimentell

In [ ]:
"""
import plotly.express as px
px.imshow([[pixel_to_plot(x, y) for x in range(w)] for y in range(h)]).show()

def erhöhe_kontrast(min_helligkeit, max_helligkeit):
    for y in range(h):
        for x in range(h):
            r, g, b = pix[x, y]
            k = (r + g + b) / 3
            0<=k<=255
            k2 = (k-min_helligkeit) * 255 / (max_helligkeit-min_helligkeit)
            k2 = max(0, min(255, int(k2)))
            pix[x, y] = (k2, k2, k2)
            
erhöhe_kontrast(50, 200)

px.imshow([[pixel_to_plot(x, y) for x in range(w)] for y in range(h)]).show()

histogram = [len([None for x in range(w) for y in range(h) if int(pixel_to_plot(x, y)) == k])/w/h for k in range(256)]
fig = go.Figure([go.Bar(x=list(range(256)), y=histogram)])
fig.show()
"""